In [1]:
from sentence_transformers import SentenceTransformer, util
from utils.operate_data import load_dataset as dd_load_dataset
from utils.autocomplete import yolo_utils as yolo
import pandas as pd

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

from sentence_transformers.cross_encoder import CrossEncoder


from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
import numpy as np



from tqdm import tqdm
tqdm.pandas()

%matplotlib inline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


# get data

In [2]:
# df_meta, _ = dd_load_dataset('/local_data/meta_data/', df_name = 'meta.pickle', files_df_name=None)
# df_meta.reset_index(inplace=True)

# df_prepared = pd.read_pickle('/local_data/meta_data/prepared_files.pickle')
# df_prepared = df_prepared[df_prepared.extension == '.pdf']
# df_prepared = df_prepared[df_prepared.content.agg(len) != 0]
# df_prepared = df_prepared.loc[df_prepared.groupby('doc_id')['order'].idxmax()]
# df_prepared.rename(columns={'content': 'text'}, inplace=True)

# df_content = df_prepared.merge(df_meta, on='doc_id')
# df_content = df_content[['doc_id', 'text', 'content']]

In [ ]:
# при необходимости получить суммаризацию. для экспериментов достаточно пользоваться фичёй content
# 
# df_content['predict_summary'] = df_content['text'].progress_apply(lambda row: summarizer(row, tokenizer, model_summarization))
# df_content.to_pickle('/local_data/summarization/data/df_summary_full.pickle')

In [2]:
df_content = pd.read_pickle('/local_data/summarization/data/df_summary_full.pickle')

In [3]:
# sentence model 'bert-base-nli-mean-tokens'
# model_sentence = SentenceTransformer('/local_data/models/text_matching/')
model_sentence = SentenceTransformer('cointegrated/rubert-tiny2')

model_summarization = AutoModelForSeq2SeqLM.from_pretrained( '/local_data/summarization/models/csebuetnlp_mT5_m2o_russian_crossSum/')
tokenizer = AutoTokenizer.from_pretrained('/local_data/summarization/data/csebuetnlp_mT5_m2o_russian_crossSum')

cross_encoder_model = CrossEncoder('cross-encoder/stsb-distilroberta-base')

In [4]:
# def summarizer(text: str, tokenizer: AutoTokenizer, model = AutoModelForSeq2SeqLM) -> str:
    
#     inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").input_ids
#     outputs = model.generate(inputs, max_new_tokens=200, do_sample=False)
    
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

## get embeddings with sentence transformer

In [712]:
%%time
# FROM CONTENT
corpus = df_content['content'].to_list()
corpus_embeddings = model_sentence.encode(corpus, convert_to_tensor=True)

CPU times: user 57.3 s, sys: 389 ms, total: 57.7 s
Wall time: 3.58 s


In [8]:
%%time
#  FROM PREDICT SUMMARY
corpus = df_content['predict_summary'].to_list()
corpus_embeddings = model_sentence.encode(corpus, convert_to_tensor=True)

CPU times: user 47.2 s, sys: 412 ms, total: 47.6 s
Wall time: 2.95 s


## matching text

In [50]:
def summarizer(text: str, tokenizer: AutoTokenizer, model: AutoModelForSeq2SeqLM) -> str:
    
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_new_tokens=200, do_sample=False)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def get_summary_query(doc: pd.DataFrame, tokenizer: AutoTokenizer,
                        model_summarization:AutoModelForSeq2SeqLM,                        
                        search_by_feature='text') -> list:
    query = doc[search_by_feature]
    summary = [summarizer(query, tokenizer, model_summarization)]
    return summary
    

def get_embedding(query: list, model_sentence: SentenceTransformer,):
    embeddings_query = model_sentence.encode(query, convert_to_tensor=True)
    return embeddings_query

def matching_text(emb_query, corpus_embeddings, top_k=5) -> dict:
    top = util.semantic_search(emb_query, corpus_embeddings, score_function=util.cos_sim, top_k=top_k)[0]
    matching = {}
    # matching = []
    for doc in top:
        doc_id = df_content.iloc[doc['corpus_id']]['doc_id']
        # matching.append(doc_id)
        matching[doc_id] = doc['score'] 
    return matching

def search_matching_text(doc: pd.DataFrame, 
                         corpus_embedings,
                         tokenizer: AutoTokenizer, 
                         model_summarization:AutoModelForSeq2SeqLM, 
                         model_sentence: SentenceTransformer,
                         search_by_feature='text', # or "content" or "predict_summary"
                         top_k= 5):
    if search_by_feature == 'text':
        summary = get_summary_query(doc, tokenizer, model_summarization, search_by_feature)
    else:
        summary = [doc[search_by_feature]]
    emb = get_embedding(summary, model_sentence)
    matching_doc_id = matching_text(emb, corpus_embeddings, top_k=top_k)
    return matching_doc_id

In [10]:
df_content.shape[0]

8058

# Encoder cross-encoder/stsb-distilroberta-base

In [673]:
# при поиске по фиче content
# 4 - 0.9, 88 - 0.86, 476 - 0.9, 50 - 0.7

# при поиске по фиче predict_summary
# 4 -0.75 , 88 - 0.75, 476 - 0.85 , 50 - 0.75

In [19]:
%%time
feature = 'predict_summary'
# feature = 'content'
ind = 476
threshold = 0.85
top_k = df_content.shape[0]

doc = df_content.iloc[ind]

query = doc[feature]
query_doc_id = df_content.iloc[ind]['doc_id']


matching_doc_id_corpus = search_matching_text(doc, 
                     corpus_embeddings, 
                     tokenizer, 
                     model_summarization, 
                     model_sentence, 
                     search_by_feature=feature,
                     top_k = top_k )

CPU times: user 413 ms, sys: 0 ns, total: 413 ms
Wall time: 170 ms


In [20]:
%%time
matching_content_corpus = {}

for key, cosin in matching_doc_id_corpus.items():
    
    sentence = df_content[df_content['doc_id'] == key][feature].values[0]
    
    
    # sentence_combinations = [query, sentence]
    # embeddings = cross_encoder_model.encode(sentence_combinations)
    # emb_query = embeddings[0]
    # emb_sentence = embeddings[1]
    # sim = util.semantic_search(emb_query, emb_sentence, score_function=util.cos_sim)[0][0]['score']
    
    sentence_combinations = [[query, sentence]]
    sim = cross_encoder_model.predict(sentence_combinations)[0]
    acc = (sim + cosin)/2
    print(sim, cosin, acc)
    if acc < threshold:
        break
    matching_content_corpus[key] = (sentence, acc)

0.8710957 1.0 0.9355478584766388
0.815468 0.9684476256370544 0.8919578194618225
0.82328004 0.957273006439209 0.8902765214443207
0.82328004 0.957273006439209 0.8902765214443207
0.8467499 0.956679105758667 0.9017145037651062
0.9236951 0.9559423923492432 0.9398187398910522
0.9236951 0.9559423923492432 0.9398187398910522
0.9171931 0.9318897128105164 0.9245414137840271
0.9179051 0.9277773499488831 0.9228412210941315
0.90970993 0.9203103184700012 0.9150101244449615
0.92578846 0.9131268262863159 0.9194576442241669
0.88397926 0.9117605686187744 0.8978699147701263
0.8404373 0.910472571849823 0.8754549324512482
0.88460606 0.8952593803405762 0.8899327218532562
0.9035543 0.8873679637908936 0.8954611420631409
0.84777707 0.8838639855384827 0.8658205270767212
0.82392323 0.8820608258247375 0.8529920279979706
0.8795894 0.8819332122802734 0.8807612955570221
0.8642553 0.8726314306259155 0.8684433698654175
0.8322481 0.872173011302948 0.8522105515003204
0.8152374 0.8720320463180542 0.8436347246170044
CPU t

In [21]:
len(matching_content_corpus)

20

In [27]:
# for key, values in matching_content_corpus.items():
#     print(values[1],values[0])

In [28]:
# matching_content_corpus

In [29]:
# yolo.show_pdf(query_doc_id)

In [30]:
# yolo.show_pdf('11c51884-1268-458e-adb6-7ea514d3df02')

# Encoder  stsb-xlm-r-multilingual

In [38]:
def summarizer(text: str, tokenizer: AutoTokenizer, model: AutoModelForSeq2SeqLM) -> str:
    
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_new_tokens=200, do_sample=False)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def get_summary_query(doc: pd.DataFrame, tokenizer: AutoTokenizer,
                        model_summarization:AutoModelForSeq2SeqLM,                        
                        search_by_feature='text') -> list:
    query = doc[search_by_feature]
    summary = [summarizer(query, tokenizer, model_summarization)]
    return summary
    
def get_embedding(query: str, cross_encoder_model: SentenceTransformer):
    embeddings_query = cross_encoder_model.encode([query], convert_to_tensor=True)[0]
    return embeddings_query

def matching_text(emb_query, corpus_embeddings, top_k=5) -> dict:
    top = util.semantic_search(emb_query, corpus_embeddings, score_function=util.cos_sim, top_k=top_k)[0]
    matching = {}
    # matching = []
    for doc in top:
        doc_id = df_content.iloc[doc['corpus_id']]['doc_id']
        # matching.append(doc_id)
        matching[doc_id] = doc['score'] 
    return matching



def search_matching_text(doc: pd.DataFrame, 
                         corpus_embedings,
                         tokenizer: AutoTokenizer, 
                         model_summarization:AutoModelForSeq2SeqLM, 
                         model_sentence: SentenceTransformer,
                         search_by_feature='text', # or "content" or "predict_summary"
                         top_k= 5):
    if search_by_feature == 'text':
        summary = get_summary_query(doc, tokenizer, model_summarization, search_by_feature)
    elif (search_by_feature == 'predict_summary') or (search_by_feature == 'content'):
        summary = doc[search_by_feature]
        emb = get_embedding(summary, model_sentence)
        matching_doc_id = matching_text(emb, corpus_embeddings, top_k=top_k)
    else:
        raise KeyError(f'"{search_by_feature}" is not matching feature. Feature must be "text" or "content" or "predict_summary"')

    return matching_doc_id

def get_similar_documents(doc: pd.DataFrame, 
                         corpus_embedings,
                         data: pd.DataFrame,
                         tokenizer: AutoTokenizer, 
                         model_summarization:AutoModelForSeq2SeqLM, 
                         semantic_model: SentenceTransformer,
                         search_by_feature='text', # or "content" or "predict_summary"
                         top_k= 5,
                         threshold=0.85):

    matching_doc_id_corpus = search_matching_text(doc, 
                     corpus_embeddings, 
                     tokenizer, 
                     model_summarization, 
                     semantic_model, 
                     search_by_feature=feature,
                     top_k = top_k )
    
    similar_documents = {key:(value, data[data['doc_id'] == key][search_by_feature].values[0]) 
                         for key, value in matching_doc_id_corpus.items() if value >= threshold}
    return similar_documents

In [12]:
# cross_encoder_model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')
cross_encoder_model = SentenceTransformer.load('/local_data/models/text_matching/cross_encoder/')

In [11]:
# cross_encoder_model.save('/local_data/models/text_matching/cross_encoder/')

In [ ]:
# %%time
# # FROM CONTENT

# corpus = df_content['content'].to_list()
# corpus_embeddings = cross_encoder_model.encode(corpus, convert_to_tensor=True)

In [13]:
%%time
# FROM PREDICT SUMMARY

corpus = df_content['predict_summary'].to_list()
corpus_embeddings = cross_encoder_model.encode(corpus, convert_to_tensor=True)

CPU times: user 24min 28s, sys: 3.65 s, total: 24min 32s
Wall time: 1min 32s


In [71]:
feature = 'predict_summary'
# feature = 'content'
ind = 476
threshold = 0.7
top_k = df_content.shape[0]

doc = df_content.iloc[ind]

query = doc[feature]
query_doc_id = df_content.iloc[ind]['doc_id']

params = {'doc': doc, 
          'corpus_embedings': corpus_embeddings, 
          'data': df_content,
          'tokenizer': tokenizer,
          'model_summarization': model_summarization,
          'semantic_model': cross_encoder_model,
          'search_by_feature': feature,
          'top_k': top_k,
          'threshold': threshold}

In [72]:
# TODO
# - механизи повторного поиска если не найдены схожие документы ( уменьшее threshold с шагом 0.5
# - время суммаризации

In [73]:
%%time
sim_docs = get_similar_documents(**params)

CPU times: user 863 ms, sys: 0 ns, total: 863 ms
Wall time: 377 ms


In [74]:
sim_docs

{'0f11e0ac-c847-459e-9baa-f184fa65476d': (0.9999999403953552,
  'О направлении информации о ходе рассмотрения проекта постановления правительства РФ внесении изменений положение управлении находящимися федеральной собственностью акциями акционерных обществ использования специального права участие российской федерации'),
 '11c51884-1268-458e-adb6-7ea514d3df02': (0.97755366563797,
  'О рассмотрении проекта постановления правительства РФ внесении изменений положение управлении находящимися федеральной собственностью акциями акционерных обществ использования специального права'),
 '193df14f-3fa5-4b30-8e78-6cb7ba56289b': (0.9719955325126648,
  'О проекте постановления правительства российской едерации внесении изменений олож ение управлении находящимися федеральной собственностью акциями акционерных обществ использовании специального права'),
 '3f4eb136-59aa-4bee-a841-1a651239bbe9': (0.9670549631118774,
  'О согласовании проекта постановления правительства РФ внесении изменений положение уп

In [65]:
query_doc_id

'0166cb25-8d0c-4896-907d-d6a46b1c36d6'

In [66]:
for key, values in sim_docs.items():
    print(values[1],values[0])

О создании рабочей группы для реализации механизма регуляторной гильотины 0.9999999403953552
О включении в состав рабочей группы реализации механизма регуляторной гильотины 0.9441139698028564
О реализации механизма регуляторной гильотины 0.897986114025116
О совещании по вопросу реализации механизма регуляторной гильотины 0.8899316787719727
О заседании круглого стола по поводу реализации механизма регуляторной гильотины 0.8634895086288452
О мониторинге реализации механизма регуляторной гильотины 0.8500397205352783
О направлении кандидатуры участников совещания по вопросу реализации механизма регуляторной гильотины 0.8361485004425049
О направлении списка рабочих групп реализации механизма регуляторной гильотины сферам деятельности федеральных органов исполнительной власти 0.8263578414916992
О направлении должностных лиц участия совещания по вопросу оценки регулирующего воздействия 0.8081548810005188
О направлении предложения нормативного закрепления правил порядка формирования ведения ак

In [18]:
yolo.show_pdf(query_doc_id)

In [19]:
yolo.show_pdf('aa4065e9-ac31-4044-b6d5-75c997ca8bb4')

в папке 3 pdf, какой открыть? 0
